Сначала прогоните ноутбук и посмотрите, как он работает. 

Потом заполните недостающие части и добейтесь RMSE <= 295000 на test.

Удачи!

Будем предсказывать цену дома по его характеристикам (опять недвижимость!)

In [1]:
import numpy as np
import pandas as pd

Посмотрим на данные

In [2]:
data = pd.read_csv("data/train.csv")
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
data_num = data.select_dtypes(exclude='object')  # Оставим только числовые признаки (не объекты)
# data_cat = data.select_dtypes(include='object')  # "Почините" категориальные признаки, чтобы их можно было использовать

In [4]:
data = data_num
# Вы можете использовать эту строчку, чтобы соединить числовые и "исправленные" категориальные признаки
# data = pd.concat([data_num, data_cat], axis=1)
print(f"Now we have {len(data.columns)} features. Wow!")

Now we have 38 features. Wow!


In [5]:
data = data.fillna(0)  # Заменяем все Nan на нули. Можно сделать лучше!

In [6]:
# Do not change this block
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['SalePrice'], axis=1),
    data['SalePrice'],
    test_size=0.2,
    random_state=666
)

In [7]:
# Стандартизуйте данные

In [8]:
from sklearn.linear_model import Ridge
from sklearn import metrics

lm = Ridge()  # Гребневая регрессия
lm.fit(X_train, y_train)
predicted_price = lm.predict(X_test)

print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predicted_price)))

RMSE: 36929.59525499005


Удалим коррелированные признаки

In [9]:
# Функцию честно взял из kaggle

def correlation(dataset, threshold):
    col_corr = set()  # Сюда будем записывать признаки, которые с кем-то коррелируют
    corr_matrix = dataset.corr()  
    for i in range(len(corr_matrix.columns)):  # Бежим по все признакам
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # Проверяем, что корреляция по модулю больше порога
                colname = corr_matrix.columns[i]  # И добавляем колонку
                col_corr.add(colname)
    return col_corr

corr_features = correlation(X_train, threshold=0.9)
print('correlated features: ', len(set(corr_features)) )

correlated features:  0


In [10]:
X_train = X_train.drop(labels=corr_features, axis=1)  # Удалим те признаки, которые с кем-то коррелируют
X_test = X_test.drop(labels=corr_features, axis=1)

In [11]:
lm = Ridge()
lm.fit(X_train, y_train)
predicted_price = lm.predict(X_test)

print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predicted_price)))

RMSE: 36929.59525499005


Если вы, заменили категориальные признаки на one-hot, SequentialFeatureSelector будет работать очень долго (слишком много перебирать)! 
В этом случае предлагаю воспользоваться следующими ячейками

In [12]:
from sklearn.feature_selection import SequentialFeatureSelector

# Прямой или обратный отбор признаков. Добавляем/удаляем по одному, пока метрики растут. 
# estimator - алгоритм, для которого перебираем признаки
# n_jobs - число процессов компьютера, -1 == использовать все
selector = SequentialFeatureSelector(estimator=Ridge(), n_jobs=-1, direction="backward", cv=2)  # direction: [forward, backward]
selector.fit(X_train, y_train)

# Функция get_support возвращает boolean mask, воспользуемся ей, как индексами
print(f"Features we took are {X_train.columns[selector.get_support()]}")

# Выбираем только те колонки, которые посчитали важными
X_train_important = X_train[X_train.columns[selector.get_support()]]
X_test_important = X_test[X_test.columns[selector.get_support()]]

Features we took are Index(['MSSubClass', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF', 'BsmtFullBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'GarageYrBlt',
       'GarageCars', 'WoodDeckSF', 'ScreenPorch', 'YrSold'],
      dtype='object')


In [13]:
lm = Ridge()
lm.fit(X_train_important, y_train)
predicted_price = lm.predict(X_test_important)

print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predicted_price)))

RMSE: 36437.134830399664


Почитать о SelectFromModel можно [здесь](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html)

Просто сортирует признаки в обученной модели по важности и оставляет те, которые выше порога (threshold)

In [14]:
from sklearn.feature_selection import SelectFromModel
# estimator - алгоритм, для которого перебираем признаки
# threshold="median" - выбираем те признаки, которые выше медианы по важности
selector = SelectFromModel(estimator=Ridge(), threshold="median")
selector.fit(X_train, y_train)
print(f"Features we took are {X_train.columns[selector.get_support()]}")

# Выбираем только те колонки, которые посчитали важными
X_train_important = X_train[X_train.columns[selector.get_support()]]
X_test_important = X_test[X_test.columns[selector.get_support()]]

Features we took are Index(['MSSubClass', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageCars', 'WoodDeckSF', 'ScreenPorch', 'PoolArea', 'YrSold'],
      dtype='object')


In [15]:
lm = Ridge()
lm.fit(X_train_important, y_train)
predicted_price = lm.predict(X_test_important)

print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predicted_price)))

RMSE: 42008.44440108443
